In [124]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [125]:
#librerias que se van a utilizar
import os
import numpy as np
from matplotlib import pyplot
from scipy import optimize
import pandas as pd # para el preprocesamiento
from sklearn import preprocessing
import csv
%matplotlib inline

In [126]:
#cargado de datos
df = pd.read_csv("/content/drive/MyDrive/IA/laboratorios/examenlabo/mnist_train.csv", sep=",")
#uso de diccionarios
df.drop_duplicates()
df.dropna(axis=0, thresh=2) #Elimina filas con n o más valores nulos.

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#ayuda a utilizar de manera mas eficiente los diccionarios
df["8x18"].value_counts()

8x18
0      19259
253     7646
252     3874
254     3867
255     1490
       ...  
75        44
58        44
167       39
90        31
74        25
Name: count, Length: 256, dtype: int64

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB
None


In [ ]:
# La entrada es de 12 elemento contando con x0
input_layer_size  = 785

num_labels = 10
#Se asignan los valore de X y X_test respectivamente 70 30
X = df.iloc[:59000, 1:]
X_test = df.iloc[59000:, 1:]
#Se asignan los valore de y e y_test respectivamente 70 30
y = df.iloc[:59000, df.columns.get_loc("label")]
y_test = df.iloc[59000:, df.columns.get_loc("label")]

In [ ]:
print(X.iloc[0,: ])
print(y)

1x1      0
1x2      0
1x3      0
1x4      0
1x5      0
        ..
28x24    0
28x25    0
28x26    0
28x27    0
28x28    0
Name: 0, Length: 784, dtype: int64
0        5
1        0
2        4
3        1
4        9
        ..
58995    1
58996    8
58997    6
58998    6
58999    4
Name: label, Length: 59000, dtype: int64


In [ ]:
#Se normalizan los datos para agilizar el proceso
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [ ]:
X_norm, mu, sigma = featureNormalize(X)

In [ ]:
# Se agrega la columna de 1 e usa para representar el término de sesgo en la regresión lineal o logística.
m, n = X.shape
X = X_norm

In [ ]:
#Esta función se utiliza para modelar la probabilidad de un evento en la regresión logística.
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [ ]:
#calcula el coste de la función
def lrCostFunction(theta, X, y, lambda_):

    m = y.size
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [ ]:
#Esta técnica permite entrenar un clasificador multiclase a partir de clasificadores binarios.
def oneVsAll(X, y, num_labels, lambda_):
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [ ]:
lambda_ = 0.001
all_theta = oneVsAll(X, y, num_labels, lambda_)

In [ ]:
print(all_theta)
print(all_theta.shape)

[[-4.61307844e-04  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-8.03860558e-05  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.58512136e-04  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [-9.15727950e-05  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.07094482e-04  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
(10, 785)


In [ ]:
#realiza predicciones con un modelo multiclase entrenado con la técnica
def predictOneVsAll(all_theta, X):

    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [127]:
#analizar la precisión del modelo en el conjunto de entrenamiento y comparar las predicciones con las etiquetas reales en el conjunto de prueba para evaluar su rendimiento en datos nuevos.
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X_test.copy()
print(XPrueba.shape)
XPrueba = np.concatenate([np.ones((1000, 1)), XPrueba], axis=1)
print(XPrueba.shape)
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)
print(y_test)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(p == y_test) * 100))

(59000, 784)
Precision del conjuto de entrenamiento: 88.76%
(1000, 784)
(1000, 785)
[6 8 5 7 2 3 9 2 6 2 1 2 0 8 3 8 3 0 8 7 4 9 5 0 9 7 4 9 1 6 2 7 6 1 8 6 1
 0 3 6 2 1 4 0 1 2 3 4 7 8 9 0 1 2 3 4 7 8 9 0 1 2 3 4 5 6 7 8 7 4 3 1 8 6
 1 9 2 4 0 9 9 3 7 7 9 1 8 4 7 5 8 5 3 2 2 0 5 8 6 0 3 8 1 0 3 0 4 7 4 9 2
 9 1 2 1 7 1 6 7 3 3 6 2 8 7 6 4 9 8 8 2 9 6 5 5 9 5 3 7 4 7 6 0 0 4 6 6 9
 0 1 1 3 2 1 0 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9
 4 2 0 2 1 5 9 6 5 9 3 8 0 7 1 1 8 2 0 7 9 6 1 6 7 4 9 5 7 9 8 5 6 2 4 8 8
 2 2 8 0 6 4 3 3 1 2 4 1 5 8 9 6 9 3 4 0 3 9 0 2 7 1 1 0 7 6 2 2 7 6 6 8 9
 4 8 4 8 3 7 8 1 0 4 5 8 3 0 6 8 3 7 8 5 7 8 2 3 8 6 7 5 9 0 1 2 5 6 7 5 9
 0 8 2 3 4 8 6 8 7 5 7 3 3 8 6 5 8 0 8 8 2 2 3 0 6 4 3 7 6 9 2 2 3 5 5 0 7
 2 3 8 0 0 5 1 8 8 0 8 8 6 5 6 7 5 3 5 3 0 8 2 3 4 8 8 7 8 9 0 1 2 3 8 8 2
 7 9 0 1 2 3 4 8 6 7 8 9 8 8 5 9 8 1 7 7 5 1 4 8 3 8 2 9 3 8 8 8 8 8 9 7 5
 4 3 9 2 5 7 4 8 2 3 6 7 1 8 8 2 8 7 2 8 1 1 8 5 6 4 8 4 7 3 6 8 0 3 7 8 9
 2 6 8 8 6 9 0 4